workflow:

Load the ResNet model from models.py.

Extract frames from videos at a specified rate.

Use the model to predict labels for these frames.

Map the predictions to the labels provided in your text files (category_momentsv2.txt and category_multi_momentsv2.txt).

Compile the results into a CSV file.

1.Download and Run the model.py first (from github)

2. import model class (resnet50), load model and process video

In [27]:
import os
os.chdir('/Users/wangjiji/Desktop/moments_models-2')  # Replace with the path to your models.py
from models import resnet50
import cv2
import torch
import torchvision.transforms as transforms
import pandas as pd


# Load the model
model = resnet50(pretrained=True)  # Ensure this matches your specific function to load resnet50
model.eval()

# Define the transformations
transform = transforms.Compose([
    transforms.ToPILImage(),                # Convert numpy array to PIL Image
    transforms.Resize((224, 224)),          # Resize the image to 224x224
    transforms.ToTensor(),                  # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Label mapping from your text file (adjust path as necessary)
with open('category_momentsv2.txt', 'r') as f:
    labels = {i: label.strip() for i, label in enumerate(f.readlines())}

# Directory containing the videos
directory = '/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/'

# Prepare to collect results
results = []

# Process each video file from 001 to 071
for i in range(1, 72):  # Adjust range for your video numbers
    video_number = f"{i:03}"
    video_path = f"{directory}{video_number}.mp4"
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Skipping: Could not open {video_path}")
        continue  # Skip this video file

    frame_rate = 16
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    interval = max(int(frame_count / frame_rate), 1)  # Prevents zero division

    for frame_id in range(0, frame_count, interval):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
        ret, frame = cap.read()
        if not ret:
            continue
        
        # Apply transformations and predict
        tensor = transform(frame)
        with torch.no_grad():
            outputs = model(tensor.unsqueeze(0))
            predicted_label = labels[torch.argmax(outputs).item()]

        results.append([video_number, frame_id, predicted_label])

    cap.release()

# Save results to CSV
# Creating the initial DataFrame
df = pd.DataFrame(results, columns=['Video', 'Frame', 'Label'])

# Pivoting the DataFrame
pivot_df = df.pivot(index='Video', columns='Frame', values='Label')

# Saving the pivoted DataFrame to CSV
pivot_df.to_csv('/Users/wangjiji/Desktop/video_labels.csv')


OpenCV: Couldn't read video stream from file "/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/006.mp4"


Skipping: Could not open /Users/wangjiji/Desktop/Social touch deep learning project/STMP4/006.mp4


OpenCV: Couldn't read video stream from file "/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/009.mp4"


Skipping: Could not open /Users/wangjiji/Desktop/Social touch deep learning project/STMP4/009.mp4


OpenCV: Couldn't read video stream from file "/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/021.mp4"


...

Skipping: Could not open /Users/wangjiji/Desktop/Social touch deep learning project/STMP4/046.mp4


OpenCV: Couldn't read video stream from file "/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/053.mp4"


Skipping: Could not open /Users/wangjiji/Desktop/Social touch deep learning project/STMP4/053.mp4


OpenCV: Couldn't read video stream from file "/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/058.mp4"


Skipping: Could not open /Users/wangjiji/Desktop/Social touch deep learning project/STMP4/058.mp4


Now, we try resnet3D50

blow is the full code of "models.py", in case

In [7]:
import os
from functools import partial

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms


def conv3x3x3(in_planes, out_planes, stride=1):
    """3x3x3 convolution with padding."""
    return nn.Conv3d(
        in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False
    )


def downsample_basic_block(x, planes, stride):
    out = F.avg_pool3d(x, kernel_size=1, stride=stride)
    zero_pads = torch.Tensor(
        out.size(0), planes - out.size(1),
        out.size(2), out.size(3), out.size(4)).zero_()
    if isinstance(out.data, torch.cuda.FloatTensor):
        zero_pads = zero_pads.cuda()
    out = torch.cat([out.data, zero_pads], dim=1)
    return out


class BasicBlock(nn.Module):
    expansion = 1
    Conv3d = staticmethod(conv3x3x3)

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = self.Conv3d(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm3d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = self.Conv3d(planes, planes)
        self.bn2 = nn.BatchNorm3d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4
    Conv3d = nn.Conv3d

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = self.Conv3d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm3d(planes)
        self.conv2 = self.Conv3d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm3d(planes)
        self.conv3 = self.Conv3d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm3d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)
        return out


class ResNet3D(nn.Module):

    Conv3d = nn.Conv3d

    def __init__(self, block, layers, shortcut_type='B', num_classes=305):
        self.inplanes = 64
        super(ResNet3D, self).__init__()
        self.conv1 = self.Conv3d(3, 64, kernel_size=7, stride=(1, 2, 2), padding=(3, 3, 3), bias=False)
        self.bn1 = nn.BatchNorm3d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0], shortcut_type)
        self.layer2 = self._make_layer(block, 128, layers[1], shortcut_type, stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], shortcut_type, stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], shortcut_type, stride=2)
        self.avgpool = nn.AdaptiveAvgPool3d(1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        self.init_weights()

    def _make_layer(self, block, planes, blocks, shortcut_type, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            if shortcut_type == 'A':
                downsample = partial(
                    downsample_basic_block,
                    planes=planes * block.expansion,
                    stride=stride,
                )
            else:
                downsample = nn.Sequential(
                    self.Conv3d(
                        self.inplanes,
                        planes * block.expansion,
                        kernel_size=1,
                        stride=stride,
                        bias=False,
                    ),
                    nn.BatchNorm3d(planes * block.expansion),
                )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, self.Conv3d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
            elif isinstance(m, nn.BatchNorm3d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)

        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


def modify_resnets(model):
    # Modify attributs
    model.last_linear, model.fc = model.fc, None

    def features(self, input):
        x = self.conv1(input)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x

    def logits(self, features):
        x = self.avgpool(features)
        x = x.view(x.size(0), -1)
        x = self.last_linear(x)
        return x

    def forward(self, input):
        x = self.features(input)
        x = self.logits(x)
        return x

    # Modify methods
    setattr(model.__class__, 'features', features)
    setattr(model.__class__, 'logits', logits)
    setattr(model.__class__, 'forward', forward)
    return model


ROOT_URL = 'http://moments.csail.mit.edu/moments_models'
weights = {
    'resnet50': 'moments_v2_RGB_resnet50_imagenetpretrained.pth.tar',
    'resnet3d50': 'moments_v2_RGB_imagenet_resnet3d50_segment16.pth.tar',
    'multi_resnet3d50': 'multi_moments_v2_RGB_imagenet_resnet3d50_segment16.pth.tar',
}


def load_checkpoint(weight_file):
    if not os.access(weight_file, os.W_OK):
        weight_url = os.path.join(ROOT_URL, weight_file)
        os.system('wget ' + weight_url)
    checkpoint = torch.load(weight_file, map_location=lambda storage, loc: storage)  # Load on cpu
    return {str.replace(str(k), 'module.', ''): v for k, v in checkpoint['state_dict'].items()}


def resnet50(num_classes=305, pretrained=True):
    model = models.__dict__['resnet50'](num_classes=num_classes)
    if pretrained:
        model.load_state_dict(load_checkpoint(weights['resnet50']))
    model = modify_resnets(model)
    return model


def resnet3d50(num_classes=305, pretrained=True, **kwargs):
    """Constructs a ResNet3D-50 model."""
    model = modify_resnets(ResNet3D(Bottleneck, [3, 4, 6, 3], num_classes=num_classes, **kwargs))
    if pretrained:
         model.load_state_dict(load_checkpoint(weights['resnet3d50']))
    return model


def multi_resnet3d50(num_classes=292, pretrained=True, **kwargs):
    """Constructs a ResNet3D-50 model."""
    model = modify_resnets(ResNet3D(Bottleneck, [3, 4, 6, 3], num_classes=num_classes, **kwargs))
    if pretrained:
        model.load_state_dict(load_checkpoint(weights['multi_resnet3d50']))
    return model


def load_model(arch):
    model = {'resnet3d50': resnet3d50,
             'multi_resnet3d50': multi_resnet3d50, 'resnet50': resnet50}.get(arch, 'resnet3d50')()
    model.eval()
    return model


def load_transform():
    """Load the image transformer."""
    return transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])])


def load_categories(filename):
    """Load categories."""
    with open(filename) as f:
        return [line.rstrip() for line in f.readlines()]


start labeling

In [24]:
import cv2
import torch
import numpy as np
import pandas as pd
from pathlib import Path
import os
os.chdir('/Users/wangjiji/Desktop/moments_models-2')  # Replace with the path to your models.py
from models import resnet3d50

# Initialize the model with pretrained weights
model = resnet3d50(num_classes=305, pretrained=True)
model.eval()

# Define the path to the label file
label_file_path = '/Users/wangjiji/Desktop/moments_models-2/category_multi_momentsv2.txt'  # Adjust this path as necessary

# Load label mapping from the text file
with open(label_file_path, 'r') as f:
    labels = {i: label.strip() for i, label in enumerate(f.readlines())}

# Define the transformation and preprocessing
def preprocess(frame):
    # Assuming frame is a numpy array of shape (H, W, C)
    return cv2.resize(frame, (112, 112))  # Resize frame to the expected input size of the model

# Directory containing the videos
directory = Path('/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/')
results = []

# Process each video file from 001 to 071
for i in range(1, 72):  # Adjust range for your video numbers
    video_number = f"{i:03}"
    video_path = directory / f"{video_number}.mp4"
    
    cap = cv2.VideoCapture(str(video_path))
    frames = []
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = preprocess(frame)  # Preprocess the frame
        frames.append(frame)
        if len(frames) == 8:  # Collect 8 frames per input sequence
            clip = np.stack(frames, axis=0)
            clip = np.transpose(clip, (3, 0, 1, 2))  # Reorder dimensions to C, D, H, W
            clip = torch.tensor(clip, dtype=torch.float32).unsqueeze(0)  # Convert to tensor and add batch dimension
            
            with torch.no_grad():
                outputs = model(clip)
                predicted_label = labels[outputs.argmax(dim=1).item()]  # Use the labels dictionary for mapping

            results.append([video_path.name, frame_count, predicted_label])
            frames = []  # Reset frames for next batch
            frame_count += 16  # Increment frame index

    cap.release()

# Convert results to DataFrame and pivot
df = pd.DataFrame(results, columns=['Video', 'Frame', 'Label'])
pivot_df = df.pivot(index='Video', columns='Frame', values='Label')
pivot_df.to_csv('/Users/wangjiji/Desktop/video_labels_resnet3D50.csv')


OpenCV: Couldn't read video stream from file "/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/006.mp4"
OpenCV: Couldn't read video stream from file "/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/009.mp4"
OpenCV: Couldn't read video stream from file "/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/021.mp4"
OpenCV: Couldn't read video stream from file "/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/023.mp4"
OpenCV: Couldn't read video stream from file "/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/031.mp4"
OpenCV: Couldn't read video stream from file "/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/032.mp4"
OpenCV: Couldn't read video stream from file "/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/034.mp4"
OpenCV: Couldn't read video stream from file "/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/036.mp4"
OpenCV: Couldn't read video stream from 

Finally we try multi_resnet3d50

In [26]:

from models import multi_resnet3d50  # Import the function for initializing the multi_resnet3d50 model

# Initialize the model with pretrained weights
model = multi_resnet3d50(num_classes=292, pretrained=True)  # Ensure that num_classes aligns with the output layer of your model
model.eval()

# Define the path to the label file
label_file_path = '/Users/wangjiji/Desktop/moments_models-2/category_multi_momentsv2.txt'  # Adjust this path as necessary

# Load label mapping from the text file
with open(label_file_path, 'r') as f:
    labels = {i: label.strip() for i, label in enumerate(f.readlines())}

# Define the transformation and preprocessing
def preprocess(frame):
    # Assuming frame is a numpy array of shape (H, W, C)
    return cv2.resize(frame, (112, 112))  # Resize frame to the expected input size of the model

# Directory containing the videos
directory = Path('/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/')
results = []

# Process each video file from 001 to 071
for i in range(1, 72):  # Adjust range for your video numbers
    video_number = f"{i:03}"
    video_path = directory / f"{video_number}.mp4"
    
    cap = cv2.VideoCapture(str(video_path))
    frames = []
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = preprocess(frame)  # Preprocess the frame
        frames.append(frame)
        if len(frames) == 8:  # Collect 8 frames per input sequence
            clip = np.stack(frames, axis=0)
            clip = np.transpose(clip, (3, 0, 1, 2))  # Reorder dimensions to C, D, H, W
            clip = torch.tensor(clip, dtype=torch.float32).unsqueeze(0)  # Convert to tensor and add batch dimension
            
            with torch.no_grad():
                outputs = model(clip)
                predicted_label = labels[outputs.argmax(dim=1).item()]  # Use the labels dictionary for mapping

            results.append([video_path.name, frame_count, predicted_label])
            frames = []  # Reset frames for next batch
            frame_count += 16  # Increment frame index

    cap.release()

# Convert results to DataFrame and pivot
df = pd.DataFrame(results, columns=['Video', 'Frame', 'Label'])
pivot_df = df.pivot(index='Video', columns='Frame', values='Label')
pivot_df.to_csv('/Users/wangjiji/Desktop/video_labels_multi_resnet3D50.csv')


OpenCV: Couldn't read video stream from file "/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/006.mp4"
OpenCV: Couldn't read video stream from file "/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/009.mp4"
OpenCV: Couldn't read video stream from file "/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/021.mp4"
OpenCV: Couldn't read video stream from file "/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/023.mp4"
OpenCV: Couldn't read video stream from file "/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/031.mp4"
OpenCV: Couldn't read video stream from file "/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/032.mp4"
OpenCV: Couldn't read video stream from file "/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/034.mp4"
OpenCV: Couldn't read video stream from file "/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/036.mp4"
OpenCV: Couldn't read video stream from 